In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
inter_df = pd.read_csv('./국토교통부_건축물대장_총괄표제부/wbdinfo_baseline_maindata_inter_checpoint_1.csv')
print(inter_df.shape)
inter_df.head()

(45376, 43)


c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
0,11650-206,집합,총괄표제부,구대장,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.0,2,0,0.0,0,0.0,0,0.00,2,30.00,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 911-29,8,일반,NaN,NaN,20100417,20171118,8
1,11650-217,집합,총괄표제부,신대장,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.0,0,0,0.0,0,0.0,110,4522.93,0,0.00,20030416.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 서초동 1496-26,18,일반,20030416.0,20030416.0,20100417,20161220,18
2,11650-397,집합,총괄표제부,신대장,1,884.0,413.10,46.73,3695.03,2346.09,265.39,공동주택,공동주택(아파트),2,0,0.0,28,0,0.0,0,0.0,28,1305.56,0,0.00,20030514.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 1-7,16,일반,20030514.0,20030514.0,20100417,20171118,16
3,11650-410,집합,총괄표제부,구대장,1,380.5,190.20,49.98,570.60,380.40,99.97,공동주택,공동주택,2,0,0.0,3,0,0.0,0,0.0,0,0.00,3,41.25,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 반포동 577-73,12,일반,NaN,NaN,20100417,20100417,12
4,11650-43,집합,총괄표제부,신대장,0,493.0,292.30,59.29,1234.89,1095.26,222.16,공동주택,"공동주택,다세대주택",2,0,0.0,12,0,0.0,0,0.0,12,0.00,0,0.00,20030925.0,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 919-10,15,일반,20030925.0,20030925.0,20100417,20171118,15


In [9]:
inter_df = inter_df.sort_values(['지번주소', '기준년월']).reset_index(drop=True)
inter_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
0,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20181114,201812,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29
1,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20190306,201912,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29
2,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20190306,202011,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29
3,11680-103,집합,총괄표제부,구대장,0,0.0,15324.37,0.00,223939.0,211555.54,0.00,공동주택,주거시설 근린생활시설,27,3,64.26,0,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20120822,201412,서울특별시 강남구 개포동 12-0,4199,일반,NaN,NaN,20120822,20200824,4199
4,11680-103,집합,총괄표제부,구대장,0,0.0,15324.37,0.00,223939.0,211555.54,0.00,공동주택,주거시설 근린생활시설,27,3,64.26,0,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20151107,201512,서울특별시 강남구 개포동 12-0,4199,일반,NaN,NaN,20120822,20200824,4199


In [3]:
inter_df['주_용도'].value_counts()

공동주택          41738
단독주택           1251
업무시설            516
제1종근린생활시설       363
제2종근린생활시설       195
공장              152
                112
노유자시설            55
숙박시설             34
종교시설             33
자동차관련시설          28
교육연구시설           23
창고시설             20
위험물저장및처리시설       16
교육연구및복지시설        10
판매시설              7
문화및집회시설           4
근린생활시설            3
Name: 주_용도, dtype: int64

In [4]:
inter_df['총세대수'].describe()

count    45376.000000
mean       226.825877
std        473.288844
min          0.000000
25%         16.000000
50%         29.000000
75%        239.000000
max       6864.000000
Name: 총세대수, dtype: float64

In [6]:
zero_df = inter_df[inter_df['총세대수'] == 0]
zero_df.shape

(1242, 43)

In [3]:
hannam_df = inter_df[inter_df['지번주소'].str.contains('한남동 810')]
hannam_df

,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
3643,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20131015,201412,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
10560,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20131015,201512,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
14671,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20131015,201612,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
19342,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20171205,201712,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
26823,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20171205,201812,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
31727,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1.0,40.45,86,1.0,88,NaN,0,20171205,201912,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600
38767,11170-100189272,집합,총괄표제부,신대장,0,111582.5,32399.477,29.04,211772.152,134423.693,120.47,공동주택,"아파트, 연립주택",33,10,66914.146,1732,0,0.0,0,0.0,1730,61335.552,2,24.0,20110107.0,1,40.45,86,1.0,88,NaN,0,20171205,202011,서울특별시 용산구 한남동 810-0,600,일반,20110107.0,20110107.0,20131015,20171205,600


In [10]:
nodupes_df = inter_df.drop_duplicates(subset=['지번주소', '기준년월'])
nodupes_df.shape

(44963, 43)

In [12]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_3.csv')
print(main_data_df.shape)

(4441256, 18)


In [13]:
nodupes_df['년'] = nodupes_df['기준년월'].apply(lambda x: int(str(x)[:4]))

In [15]:
merge_df = main_data_df.merge(nodupes_df, on=['지번주소', '년'])
merge_df.shape

(1539528, 60)

In [16]:
merge2_df = merge_df[(merge_df['전월세매매구분'] == '매매')|(merge_df['전월세매매구분'] == '전세')]
merge2_df.shape

(1249708, 60)

In [17]:
land_specs_df = pd.read_csv('./prepped_data/land_specs_ver_3.csv')
print(land_specs_df.shape)
land_specs_df.head()

(1563645, 11)


,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
0,서울특별시 강남구 개포동 1163-4,2006,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
1,서울특별시 강남구 개포동 1163-4,2007,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
2,서울특별시 강남구 개포동 1163-4,2008,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
3,서울특별시 강남구 개포동 1163-4,2009,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0
4,서울특별시 강남구 개포동 1163-4,2010,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,6290000.0


In [18]:
hannam_df = land_specs_df[land_specs_df['지번주소'].str.contains('한남동 810')]
hannam_df.head()

,지번주소,년,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가
1312110,서울특별시 용산구 한남동 810-0,2006,대,111582.5,제2종일반주거지역,제1종일반주거지역,아파트,완경사,부정형,중로한면,5500000.0
1312111,서울특별시 용산구 한남동 810-0,2007,대,111582.5,제2종일반주거지역,제1종일반주거지역,아파트,완경사,부정형,중로한면,5500000.0
1312112,서울특별시 용산구 한남동 810-0,2008,대,111582.5,제2종일반주거지역,제1종일반주거지역,아파트,완경사,부정형,중로한면,5500000.0
1312113,서울특별시 용산구 한남동 810-0,2009,대,111582.5,제2종일반주거지역,제1종일반주거지역,아파트,완경사,부정형,중로한면,5500000.0
1312114,서울특별시 용산구 한남동 810-0,2010,대,111582.5,제2종일반주거지역,제1종일반주거지역,아파트,완경사,부정형,중로한면,5500000.0


In [21]:
inter_df['기준년월_최초사용승인일'].fillna(inter_df['주소별_최초생성일자'], inplace=True)

In [22]:
inter_df['사용승인년도'] = inter_df['기준년월_최초사용승인일'].apply(lambda x: int(str(x)[:4]))
inter_df['사용승인년도'].describe()

count    45376.000000
mean      2009.574312
std          6.888021
min       1903.000000
25%       2009.000000
50%       2011.000000
75%       2013.000000
max       2020.000000
Name: 사용승인년도, dtype: float64

In [26]:
date_df = inter_df[['지번주소', '기준년월', '사용승인년도']].drop_duplicates(subset=['지번주소', '기준년월'])
date_df.shape

(44963, 3)

In [27]:
date_df.to_csv('./국토교통부_건축물대장_표제부/approv_date.csv', index=False)

In [25]:
inter_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합,사용승인년도
0,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20181114,201812,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29,2018
1,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20190306,201912,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29,2018
2,11680-100276255,집합,총괄표제부,신대장,0,529.1,305.09,57.66,1055.1,1055.10,199.41,공동주택,도시형생활주택(단지형다세대주택),2,0,0.00,15,0,0.0,0,0.0,15,0.0,0,0.0,20181112.0,NaN,0.0,76,NaN,0,NaN,0,20190306,202011,서울특별시 강남구 개포동 1193-5,29,일반,20181112.0,20181112.0,20181114,20190306,29,2018
3,11680-103,집합,총괄표제부,구대장,0,0.0,15324.37,0.00,223939.0,211555.54,0.00,공동주택,주거시설 근린생활시설,27,3,64.26,0,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20120822,201412,서울특별시 강남구 개포동 12-0,4199,일반,20120822.0,NaN,20120822,20200824,4199,2012
4,11680-103,집합,총괄표제부,구대장,0,0.0,15324.37,0.00,223939.0,211555.54,0.00,공동주택,주거시설 근린생활시설,27,3,64.26,0,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20151107,201512,서울특별시 강남구 개포동 12-0,4199,일반,20120822.0,NaN,20120822,20200824,4199,2012


In [28]:
inter_df['총세대수'].describe()

count    45376.000000
mean       226.825877
std        473.288844
min          0.000000
25%         16.000000
50%         29.000000
75%        239.000000
max       6864.000000
Name: 총세대수, dtype: float64

In [29]:
sub_df = inter_df[['지번주소', '기준년월', '부속_건축물_수']]

In [30]:
sub_df['부속건축물수'] = sub_df.groupby(['지번주소', '기준년월']).transform('max')['부속_건축물_수']

In [31]:
sub_df.drop(columns=['부속_건축물_수'], inplace=True)
sub_df.drop_duplicates(inplace=True)

In [32]:
sub_df.shape

(44963, 3)

In [33]:
sub_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_부속건축물수.csv', index=False)

In [34]:
lot_num_df = inter_df[['지번주소', '기준년월', '외필지_수']]

In [35]:
lot_num_df['외필지수'] = lot_num_df.groupby(['지번주소', '기준년월']).transform('max')['외필지_수']
lot_num_df = lot_num_df.drop(columns=['외필지_수']).drop_duplicates()
lot_num_df.shape

(44963, 3)

In [36]:
lot_num_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_외필지수.csv', index=False)

In [37]:
parking_df = inter_df[['지번주소', '기준년월', '총_주차_수']]
parking_df['총주차수'] = parking_df.groupby(['지번주소', '기준년월']).transform('max')['총_주차_수']
parking_df = parking_df.drop(columns=['총_주차_수']).drop_duplicates()
parking_df.shape

(44963, 3)

In [38]:
parking_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_총주차수.csv', index=False)

In [40]:
epi_df = inter_df[['지번주소', '기준년월', '에너지_EPI점수']]
epi_df['epi'] = epi_df.groupby(['지번주소', '기준년월']).transform('max')['에너지_EPI점수']
epi_df = epi_df.drop(columns=['에너지_EPI점수']).drop_duplicates()
epi_df.shape

(44963, 3)

In [41]:
epi_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_epi점수.csv', index=False)

In [42]:
enviro_df = inter_df[['지번주소', '기준년월', '친환경_건축물_인증점수']]
enviro_df['enviro'] = enviro_df.groupby(['지번주소', '기준년월']).transform('max')['친환경_건축물_인증점수']
enviro_df = enviro_df.drop(columns=['친환경_건축물_인증점수']).drop_duplicates()
enviro_df.shape

(44963, 3)

In [43]:
enviro_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_친환경점수.csv', index=False)

In [44]:
intelli_df = inter_df[['지번주소', '기준년월', '지능형_건축물_인증점수']]
intelli_df['intelli'] = intelli_df.groupby(['지번주소', '기준년월']).transform('max')['지능형_건축물_인증점수']
intelli_df = intelli_df.drop(columns=['지능형_건축물_인증점수']).drop_duplicates()
intelli_df.shape

(44963, 3)

In [45]:
intelli_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_지능형점수.csv', index=False)

In [46]:
inter_df['기계식수'] = inter_df['옥내_기계식_대수(대)'] + inter_df['옥외_기계식_대수(대)']
mecha_df = inter_df[['지번주소', '기준년월', '기계식수']]
mecha_df['총기계식수'] = mecha_df.groupby(['지번주소', '기준년월']).transform('max')['기계식수']
mecha_df = mecha_df.drop(columns=['기계식수']).drop_duplicates()
mecha_df.shape

(44963, 3)

In [47]:
mecha_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_총기계식주차수.csv', index=False)

In [48]:
sub_area_df = inter_df[['지번주소', '기준년월', '부속_건축물_면적(㎡)']]
sub_area_df['부속건축물면적'] = sub_area_df.groupby(['지번주소', '기준년월']).transform('max')['부속_건축물_면적(㎡)']
sub_area_df = sub_area_df.drop(columns=['부속_건축물_면적(㎡)']).drop_duplicates()
sub_area_df.shape

(44963, 3)

In [49]:
sub_area_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_부속건축물면적.csv', index=False)

In [50]:
area_df = inter_df[['지번주소', '기준년월', '연면적(㎡)']]
area_df['연면적'] = area_df.groupby(['지번주소', '기준년월']).transform('max')['연면적(㎡)']
area_df = area_df.drop(columns=['연면적(㎡)']).drop_duplicates()
area_df.shape

(44963, 3)

In [51]:
area_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_연면적.csv', index=False)

In [52]:
housenum_df = inter_df[['지번주소', '기준년월', '총세대수']]
housenum_df['총세대수'] = housenum_df.groupby(['지번주소', '기준년월']).transform('max')['총세대수']
housenum_df = housenum_df.drop_duplicates()
housenum_df.shape

(44963, 3)

In [53]:
housenum_df.to_csv('./국토교통부_건축물대장_총괄표제부/총괄표제부상_총세대수.csv', index=False)

In [54]:
inter_df['주_용도'].value_counts()

공동주택          41738
단독주택           1251
업무시설            516
제1종근린생활시설       363
제2종근린생활시설       195
공장              152
                112
노유자시설            55
숙박시설             34
종교시설             33
자동차관련시설          28
교육연구시설           23
창고시설             20
위험물저장및처리시설       16
교육연구및복지시설        10
판매시설              7
문화및집회시설           4
근린생활시설            3
Name: 주_용도, dtype: int64

In [55]:
inter_df['기타_용도'].nunique()

1392

In [56]:
inter_df['기타_용도'].value_counts()[:100]

공동주택                            5419
다세대주택                           4944
공동주택(아파트)                       4776
아파트                             3511
연립주택                            3170
도시형생활주택(단지형다세대)                 1342
도시형생활주택(단지형다세대주택)                993
주택                               747
공동주택(다세대주택)                      716
공동주택,아파트                         532
다세대주택(도시형생활주택/단지형다세대)            519
공동주택(아파트), 생활편익시설                438
주거시설 근린생활시설                      326
주거시설                             320
공동주택(아파트), 부대복리시설                258
아파트, 근린생활시설                      253
도시형생활주택                          250
단독주택                             239
공동주택(아파트),근린생활시설                 230
단지형다세대주택                         216
공동주택(연립주택)                       194
다세대주택(도시형생활주택)                   189
공동주택,다세대주택                       175
공동주택(아파트),생활편익시설                 168
공동주택, 근린생활시설                     160
공동주택(도시형생활주택)                    144
아파트, 상가                          138
업